In [5]:
%reload_ext autoreload
%autoreload 2

import os.path
import sys
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
import pandas as pd

PARENT_DIR = r'D:\projects\diverters\src'
CURR_DIR = os.path.join(PARENT_DIR, 'conductivity_effect')
sys.path.append(PARENT_DIR)

import materials.sampling as sampling
import linearization.surface as surface
from materials.sampling import SampleMaterial
from parametric_solver.solver import BilinearThermalSolver, BilinearThermalSample, NodeContext

In [7]:
OUT_DIR = os.path.join(CURR_DIR, 'out', 'temp_interp', 'kdoped_rhenium')
KDOPED_RHENIUM_DIR = os.path.join(CURR_DIR, 'in', 'temp_interp', 'kdoped_rhenium')
INP_DIR = os.path.join(KDOPED_RHENIUM_DIR, 'inp')
MATPOINT_DIR = os.path.join(KDOPED_RHENIUM_DIR, 'matpoint')

CASES = ['low', 'nominal', 'high']
COMPS = ['jet_matpoint.out', 'thimble_matpoint.out']
SAMPLE_N = 9

In [ ]:
solver = BilinearThermalSolver(write_path=OUT_DIR, loglevel="INFO", nproc=8)

for i in range(1,  + 1):
    sample = BilinearThermalSample()
    sample.name = f"kdoped_rhenium_{i}"
    sample.input = os.path.join(INP_DIR, f"{sample.name}.inp")
    sampling.set_structural(sample, SampleMaterial.W_3RHENIUM, True)
    solver.add_sample(sample)

# Solve at all sample points
solver.solve(verbose=False)

In [12]:
def get_result(_i):
    return solver.result_from_name(f"kdoped_rhenium_{_i}")

def get_df(_path):
    _df = pd.read_csv(_path, index_col=0)
    _df.columns = _df.columns.str.strip()
    return _df

def mean_temp(_path):
    _df = get_df(_path)
    return _df['temperature'].mean()

def max_eqv_stresses():
    _max_stresses = []
    for _i in range(1, SAMPLE_N):
        _result = get_result(_i)
        
        if not _nodes:
            _nodes = _result.valid_nodes()

        _max_stresses.append(max([_result.eqv_stress(node) for node in _nodes]))

    return _max_stresses

def mean_temps():
    _avg_temps = []
    _avg_temps.append(mean_temp(os.path.join(MATPOINT_DIR, 'thimble_matpoint_low.out')))
    
    midpoints = int((SAMPLE_N - 3) / 2 + 1)
    for _i in range(1, midpoints):
        _path = os.path.join(MATPOINT_DIR, f'thimble_matpoint_low_{_i}.out')
        _avg_temps.append(mean_temp(_path))
        
    _avg_temps.append(mean_temp(os.path.join(MATPOINT_DIR, 'thimble_matpoint_nominal.out')))
    
    for _i in range(1, midpoints):
        _path = os.path.join(MATPOINT_DIR, f'thimble_matpoint_nominal_{_i}.out')
        _avg_temps.append(mean_temp(_path))
        
    _avg_temps.append(mean_temp(os.path.join(MATPOINT_DIR, 'thimble_matpoint_high.out')))
    
    return _avg_temps
        
def max_linearization(stress):
    NODES_DIR = os.path.join(PARENT_DIR, 'inp', 'nodes')
    TOP_SURFACE_PATH = os.path.join(NODES_DIR, 'ts.node.loc')
    BOTTOM_SURFACE_PATH = os.path.join(NODES_DIR, 'bs.node.loc')
    ALL_LOCS_PATH = os.path.join(NODES_DIR, 'all.node.loc')

    membrane_data = []
    bending_data = []
    linearized_data = []

    for _i in range(1, SAMPLE_N):
        curr_name = sample_name(_i)
        curr_result = solver.result_from_name(curr_name)

        if stress:
            dataframe = curr_result.stress_dataframe()
        else:
            dataframe = curr_result.strain_dataframe()

        stress_result = surface.linearize_surface(
            TOP_SURFACE_PATH,
            BOTTOM_SURFACE_PATH,
            dataframe,
            ALL_LOCS_PATH,
            None
        )

        total_result = stress_result['membrane'] + stress_result['bending']

        membrane_data.append(max(stress_result['membrane']))
        bending_data.append(max(stress_result['bending']))
        linearized_data.append(max(total_result))
        
    return membrane_data, bending_data, linearized_data

def plot_vals(x, y, legend, title, xlabel, ylabel ax=None):
    if ax is None:
        ax = plt.subplot()
        
    ax.plot(x, y, label=label)
    ax.scatter(x, y)
    ax.set_title(f"{prop_str} vs Temperature")
    ax.set_xlabel("Temperature (°C)")
    ax.set_ylabel(f"{prop_str} ({unit_str})")
    ax.legend()

[1062.2589836295635, 1199.266374894722, 1336.2737661598808, 1473.2811574250395, 1610.2885486901985, 1785.0437507829292, 1959.7989528756607, 2134.554154968391, 2309.309357061123]
